In [29]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GEMINI_API_Key']=os.getenv("GEMINI_API_Key")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [30]:
## Data INgestion by Langchain and webapp
from langchain_community.document_loaders import PyMuPDFLoader

In [31]:
loader=PyMuPDFLoader("18118 LEASE001.pdf")
loader

In [32]:
docs=loader.load()
docs

[Document(metadata={'source': '18118 LEASE001.pdf', 'file_path': '18118 LEASE001.pdf', 'page': 0, 'total_pages': 54, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': "D:20240326154523+05'30'", 'modDate': '', 'trapped': ''}, page_content='t\n@\nlndian-l,lon Judicial Stamp\nHaryana Govemment\nt\n@\nDale :141132023\nI101\nCe.tificale No.\nGRN No.\nG0a{202rc695\n1002 78\n$rne Duty P.id\nFadty:\nt0\nNanc :\nH.No/Floor\nCityMlhga\nPhona :\nVsd(r LE\nN.\nGrruffl\n98"\'"28\nS.ei\\rvrd : t{3\ntxri, : Guu$a|rr\nl-Ildnat : lla\nSh|!: Hdya|a\nffi\nPurpose : AGREEMENT lo ba rub,nin d at Co[c.mad onlcr\nt&ll&\nnE rlttrtry ol tB d@rrurr cr! !. y.ra.d !y lcrEE !l.3 OC.d. Iko.Jgn !6rr prD.. s cr ib r.a.r!. nto.r,bgrrtry rEn\nLEASE DEED\nThis LEASE-DEED (hereafter referred to as "Lease Deed") is executed at Gurugram on this\n- \nday of\nBY\nVatika Ltd. a Company incorporated underthe Companies Act 1956 having its registered off

In [33]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_documents(docs)
chunks


[Document(metadata={'source': '18118 LEASE001.pdf', 'file_path': '18118 LEASE001.pdf', 'page': 0, 'total_pages': 54, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': "D:20240326154523+05'30'", 'modDate': '', 'trapped': ''}, page_content='t\n@\nlndian-l,lon Judicial Stamp\nHaryana Govemment\nt\n@\nDale :141132023\nI101\nCe.tificale No.\nGRN No.\nG0a{202rc695\n1002 78\n$rne Duty P.id\nFadty:\nt0\nNanc :\nH.No/Floor\nCityMlhga\nPhona :\nVsd(r LE\nN.\nGrruffl\n98"\'"28\nS.ei\\rvrd : t{3\ntxri, : Guu$a|rr\nl-Ildnat : lla\nSh|!: Hdya|a\nffi\nPurpose : AGREEMENT lo ba rub,nin d at Co[c.mad onlcr\nt&ll&\nnE rlttrtry ol tB d@rrurr cr! !. y.ra.d !y lcrEE !l.3 OC.d. Iko.Jgn !6rr prD.. s cr ib r.a.r!. nto.r,bgrrtry rEn\nLEASE DEED\nThis LEASE-DEED (hereafter referred to as "Lease Deed") is executed at Gurugram on this\n- \nday of\nBY\nVatika Ltd. a Company incorporated underthe Companies Act 1956 having its registered off

In [34]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=os.getenv("GEMINI_API_Key"))


I0000 00:00:1723104536.070329 4205414 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [35]:
# # specify embedding model (using huggingface sentence transformer)
# from langchain.embeddings import HuggingFaceEmbeddings
# embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
# #model_kwargs = {"device": "cuda"}
# embeddings = HuggingFaceEmbeddings(
#   model_name=embedding_model_name,
#   #model_kwargs=model_kwargs
# )

In [36]:
# Vectore Store

from langchain_community.vectorstores import FAISS

vectorstoredb=FAISS.from_documents(chunks, embedding)

In [37]:
vectorstoredb

In [38]:
# Query with Vectorstore db

query="Rent of 1st year"
result=vectorstoredb.similarity_search(query)
print(result[0].page_content)

FOLLOWS:
Lease-Rental: THE Lessor hereby leases the Demased Premises unto the Lessee and the
Lessee hereby takes on lease the Demised Premises together with the right to use the common
facilities in the said Complex along with other occupants. The Lease Rental for the Demised
Premises is mentioned below for the term of three years.:
Year
Rental Per Sq. Ft.
Total Monthly Amount
1!t Year
Rs. 55 per sq. ft. (Plus GST
as Applicable)
Rs.2,35,950/- (Rupees Two
Lakh Thirty-Five Thousand
Nine Hundred and Fifly
Only) (Plus GST as
Applicable)
znd Yeat
Rs. 65 per sq. ft. (Plus GST
as Applicable)
Rs.2,78,850/- (Rupees Two
Lakh Seventy-Eight
Thousand Eight Hundred
and Fifty Only) (Plus GST as
Applicable)
3d Year
Rs. 75 per sq. ft. (Plus GST
as Applicable)
Rs. 3,21,7$/- (Rupees
Three Lakh Twenty-One
Thousand Seven Hundred
and Fifty Only) (Plus GST as
Applicable)
2. Commencement: THAT the Lease for the initial term of 3 (three) years shall commence from


In [39]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=os.getenv("GEMINI_API_Key"))


I0000 00:00:1723104580.266799 4205414 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1723104580.268439 4205414 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [40]:
# Retrival chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [41]:
tempplate =  """
You are an expert in extracting data , extract data from given context based on questions provided and provide actual answer from context:
<context>
{context}
</context>


"""


In [42]:
prompt =ChatPromptTemplate.from_template(tempplate)

# Create stuff document

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nYou are an expert in extracting data , extract data from given context based on questions provided and provide actual answer from context:\n<context>\n{context}\n</context>\n\n\n'))])
| ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x16591e170>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x166bc69b0>, default_metadata=())
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [43]:
# Create Retriver

retriver = vectorstoredb.as_retriever()

In [44]:
# retrival chain

from langchain.chains import create_retrieval_chain
retrival_chain = create_retrieval_chain(retriver, document_chain)

In [45]:
retrival_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x165815ed0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nYou are an expert in extracting data , extract data from given context based on questions provided and provide actual answer from context:\n<context>\n{context}\n</context>\n\n\n'))])
            | ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), client=<goog

In [46]:
question =""" ANswer my questions with information and try to get best one single string  with below mentioned format only
1. Lessee Name 
1. rent per in 1st year
2. rent value
3. leased square feet taken by buyer  , Super Area , Carpet area, builtup
4. Lease start date
5. Lease End date calculate by starte date if not given
6. lock in period in months if given if not give "NA"
7. Notice Period in days or months
8. Rent Escalation in details
"""

In [47]:
question =""" ANswer my questions with information and try to get best one single string  with below mentioned format only
1. rent per in 1st year
2. rent value
3. leased Area by buyer  , Super Area , Carpet area, builtup
4. Lease start date
5. Lease End date calculate by starte date if not given
6. lock in period in months if given if not give "NA"
7. Notice Period in days or months
8. Rent Escalation in details
9. Location and Floor of area leased to second party
10. lessor information and company name , and who is leasing the property
11. Lessee information and company name , and whom is leasing the property
"""

In [49]:
## Get the response form the LLM
response=retrival_chain.invoke({"input":"Provide me info"})
print(response['answer'])

AttributeError: 'int' object has no attribute 'name'

In [ ]:
## Get the response form the LLM
response=retrival_chain.invoke({"input":question})
print(response['answer'])

In [ ]:
response=retrival_chain.invoke({"input":"Name of lessor/ lessee"})
print(response['answer'])

In [ ]:
response=retrival_chain.invoke({"input":"provide single string answers and give information"})
print(response['answer'])

In [ ]:
response=retrival_chain.invoke({"input":"provide answers of given questions Name of First party , Second Party , leased location"})
print(response['answer'])

In [ ]:
response=retrival_chain.invoke({"input":"Carpet Area"})
response['answer']

In [ ]:
response=retrival_chain.invoke({"input":"date"})
response['answer']

In [ ]:
response=retrival_chain.invoke({"input":"Security amount"})
response['answer']

In [ ]:
response=retrival_chain.invoke({"input":"GSTIN"})
response['answer']

In [ ]:
print(response['answer'])